---

[//]: # "Header"

<p style="text-align: left;  font-size:18pt; LINE-HEIGHT:30px;">
    <span style="float: left">
     Technische Hochschule Ingolstadt<br>
     Prof. Dr. Sören Gröttrup<br>
     Laura Dietl
    </span>
    <span style="float: right;">
       Machine Learning 1<br>
        <span style="float: right;">WS 24/25</span>
    </span>
</p>

[//]: # "Header Aufgabenblatt"
<br>
<p style="text-align: center;  font-size:18pt; LINE-HEIGHT:30px;">
     <span style="font-weight: bold;">Aufgabenblatt 5</span><br>
     Themen: ROC<br>
     Abgabetermin: 13.12.2024, 23:59 Uhr <br>
    Punkte: 20
</p>


---

**Namen:** Emelie Hauck, Conrad Christoph

---

# 1. ROC

In [11]:
# Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
plt.style.use('ggplot')
from sklearn.metrics import roc_curve

## Aufgabe 1.1 [10 Punkte]
1. Schreiben Sie eine Funktion `calc_roc(y_labels, pred_prob)`, welche die Werte der ROC-Kurve (FPR, TPR , threshold) für ein binäre Klassifikation berechnet und diese zurückgibt. Die Eingaben `y_labels` enthält dabei die wahren Werte eine Zielvariablen und `pred_pob` ist ein Verktor mit Wahrscheinlichkeiten für die Zugehörigkeit des jeweiligen Sample zur positiven Klassen. Beispiele sind unterhalb der Aufgabe angegeben.
1. Erweitern Sie die Funktion so, dass sie zusätzlich den AUC berechnet und diesen ebenfalls zurückgibt, sprich `roc_values, auc = calc_roc(y_labels, pred_prob)`
1. Plotten Sie die ROC-Kurve für die unten agegebenen Vektoren `y_labels` und `pred_prob` und berechnen Sie die AUC
1. Bei welchen Schwellwert (Threshold) für die Entscheidung ob ein Sample zur Klasse 1 gehört ist die Accuracy am größten?

_Hinweis:_ 
* Für die Aufgabe dürfen Sie nicht die von Python (sklearn) bereitgestellten Funktionen für die Berechnung des roc und auc verwenden, sondern sollen diese selbständig entwickeln.
* Python stellt die Funktion `sklearn.metrics.roc_curve` zur Berechnung der TPR und FPR zur Verfügung. Sie können diese Funktion als Kontrolle nutzen, um die Richtigkeit Ihrer Berechnungen zu verifizieren.

In [12]:
y_labels = np.load(file="Daten Blatt 5/target.npy")
pred_prob = np.load(file="Daten Blatt 5/pred.npy")

In [137]:
#### Code HERE ####

def cal_roc(y_labels, pred_prob):
    
    thresholds = np.unique(pred_prob)
    tprs = []
    fprs = []
    x = 2
    for theta in thresholds:
        pred = (pred_prob >= theta).astype(int)

        TP = np.sum((pred == 1) & (y_labels == 1))
        FP = np.sum((pred == 1) & (y_labels == 0))
        TN = np.sum((pred == 0) & (y_labels == 0))
        FN = np.sum((pred == 0) & (y_labels == 1))
    
        TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
        FPR = FP / (FP + TN) if (FP + TN) > 0 else 0

        tprs.append(TPR)
        fprs.append(FPR)
    
    control_fprs, control_tprs, control_thresholds = roc_curve(y_labels, pred_prob)

    tprs = np.array(tprs)
    tprs = tprs[::-1]
    fprs = np.array(fprs)
    thresholds = np.array(thresholds)

    print(tprs)
    print("heyy")
    print(control_tprs)


cal_roc(y_labels, pred_prob)

[0.         0.00806452 0.01209677 0.01612903 0.01612903 0.02016129
 0.02822581 0.03225806 0.03629032 0.04435484 0.05241935 0.05645161
 0.06048387 0.06451613 0.06854839 0.07258065 0.08467742 0.08870968
 0.09274194 0.10080645 0.10483871 0.11693548 0.125      0.12903226
 0.13306452 0.13709677 0.14516129 0.14919355 0.15725806 0.16129032
 0.16129032 0.16532258 0.16935484 0.1733871  0.17741935 0.18548387
 0.18951613 0.19354839 0.2016129  0.20564516 0.20967742 0.20967742
 0.21370968 0.22177419 0.22983871 0.23387097 0.24193548 0.24596774
 0.25       0.25403226 0.25806452 0.25806452 0.26209677 0.26612903
 0.27419355 0.27822581 0.28225806 0.28629032 0.29032258 0.29032258
 0.29435484 0.2983871  0.30241935 0.30645161 0.31451613 0.32258065
 0.3266129  0.33064516 0.33467742 0.33870968 0.34274194 0.35080645
 0.35887097 0.36290323 0.36693548 0.37096774 0.375      0.37903226
 0.37903226 0.38709677 0.39112903 0.39919355 0.40322581 0.41935484
 0.4233871  0.43145161 0.43951613 0.44354839 0.44758065 0.4516

## Aufgabe 1.2 [10 Punkte]

**Daten:** Der Datensatz `creditcard_small.csv` ist ein gekürzter Datensatz mit weniger Variablen von dem Original-Datensatz https://www.kaggle.com/mlg-ulb/creditcardfraud. Er enthält anonymisierte Informationen von Kreditkartentransaktionen und ein Label `Class`, ob diese Transaktion betrügerisch war (`Class=1`), oder nicht (`Class=2`).

1. Trainieren Sie das Logistisches-Regrassionsmodell `log_reg = LogisticRegression()` (siehe Code unten) auf den Trainingsdaten `x_train, y_train` und machen Sie damit eine Vorhersage für die Testdaten `x_test`.
1. Berechnen Sie diverse Evaluationsmetriken für de Performance des Modells auf den Testdaten mittels einen Klassifikations-Report durch die Funktion `classification_report` aus dem Modul `sklearn.metrics`. Was ist die Accuracy, Precision, Sensitivity, Specificity und F1-Score für das Modell?
1. Berechnen Sie die AUC und plotten Sie die ROC-Kurve für die Testdaten. Interpretieren Sie dessen Verlauf. Sie können hier die Funktionen `sklearn.metrics.roc_curve` und `sklearn.metrics.roc_auc_score` benutzen.
1. Trainieren Sie das Neuronale Netz `nn = MLPClassifier(activation="relu")` auf den Trainigsdaten `x_train, y_train`. Wie ist die AUC bei diesem Modell auf den Testdaten? Plotten Sie auch hier die ROC-Kurve und vergleichen Sie diese mit der der Logistischen Regression.

**Vorgeschriebener Code. Bitte nicht verändern**

In [138]:
# Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
plt.style.use('ggplot')

In [139]:
#load data
data = pd.read_csv("Daten Blatt 5/creditcard_small.csv")
data

,V5,V6,V7,V8,V27,V28,Class
0,-0.338321,0.462388,0.239599,0.098698,0.133558,-0.021053,0
1,0.060018,-0.082361,-0.078803,0.085102,-0.008983,0.014724,0
2,-0.503198,1.800499,0.791461,0.247676,-0.055353,-0.059752,0
3,-0.010309,1.247203,0.237609,0.377436,0.062723,0.061458,0
4,-0.407193,0.095921,0.592941,-0.270533,0.219422,0.215153,0
...,...,...,...,...,...,...,...
284802,-5.364473,-2.606837,-4.918215,7.305334,0.943651,0.823731,0
284803,0.868229,1.058415,0.024330,0.294869,0.068472,-0.053527,0
284804,2.630515,3.031260,-0.296827,0.708417,0.004455,-0.026561,0
284805,-0.377961,0.623708,-0.686180,0.679145,0.108821,0.104533,0


In [140]:
#Split into train and test
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data, test_size=0.3, random_state=1)

In [141]:
y_train = data_train.Class
x_train = data_train.drop(columns=["Class"])

y_test = data_test.Class
x_test = data_test.drop(columns=["Class"])

In [142]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
log_reg = LogisticRegression()
nn = MLPClassifier(activation="relu")

**Ihren Code ab hier einfügen**

In [143]:
#### CODE HERE ####
...